### Pipeline for QnA with Memory

In [1]:
####Notes####
# chunked data length can be atmost 16 for azureopenai

In [2]:
# !pip3 install langchain
# !pip install openai
# !pip3 install openai chromadb
# !pip3 install tiktoken

In [3]:
# !pip3 install unstructured
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip3 install pdfminer.six
# !pip3 install pymupdf 
# !pip3 install lxml
# !pip3 install lxml

In [2]:
import os
import sys
import glob
import re
import importlib
import langchain
from pathlib import Path
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
#environment-variables
load_dotenv()
sys.path.append(os.path.join(os.getcwd(), '../scripts'))
import helpers as h
import constants as c
import scraper as sc
importlib.reload(h)
importlib.reload(c)
importlib.reload(sc)

<module 'scraper' from '/Users/rachitjoshi/Documents/Finbot/notebooks/../scripts/scraper.py'>

#### Paths

In [3]:
pdfs_path = os.path.join(os.getcwd(), '../data/pdfs')
merged_pdfs_path = os.path.join(os.getcwd(), '../data/pdfs_processed/merged')

#### Data

In [4]:
investorcom_pdfs = [ 
    _ for _ in glob.glob(os.path.join(pdfs_path, '*.pdf')) if 'investorcom' in _
]

In [5]:
# web_list = sc.scrape_site(
#     "https://www.economist.com/",
#     c.header_template
# )
# web_list

In [6]:
# web_list = [
#     'https://www.scientificamerican.com/article/pangaea-ultima-the-next-supercontinent-may-doom-mammals-to-far-future-extinction/',
#     'https://www.scientificamerican.com/article/ho-oleilana-a-billion-light-year-wide-bubble-of-galaxies-astounds-astronomers/',
#     'https://www.scientificamerican.com/article/dead-trees-and-a-mysterious-cosmic-explosion-reveal-bigger-quake-risk-for-seattle/',
#     'https://www.scientificamerican.com/article/streetlights-are-mysteriously-turning-purple-heres-why/',
#     'https://www.scientificamerican.com/article/to-move-fast-quantum-maze-solvers-must-forget-the-past/',
# ]

##### - Load

In [7]:
loaded_data = []
for pdf in investorcom_pdfs :
    loaded_data.extend(UnstructuredPDFLoader(file_path=pdf).load())        

In [8]:
# loaded_data = []
# for page in web_list[:40]:
#     try:
#         loaded_data.extend(WebBaseLoader(page).load())
#     except:
#         continue

In [9]:
len(loaded_data)

7

In [10]:
print(loaded_data[5].metadata)

{'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/meta_investorcom_1.pdf'}


In [11]:
# for pdf in pdf_list :
#     try :
#         pdf_data.extend(UnstructuredPDFLoader(file_path=pdf).load())        
#     except NameError:
#         pdf_loader = UnstructuredPDFLoader(file_path=pdf)
#         pdf_data = pdf_loader.load()

##### - Split

In [27]:
# chunk_size = c.prompt_max//c.retrieval_kwargs['k']
chunk_size = 4000
chunk_overlap = 0

In [28]:
data_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size, chunk_overlap = chunk_overlap
)

In [29]:
chunked_data = data_splitter.split_documents(loaded_data)

In [30]:
for doc in chunked_data:
    doc.metadata['company']=os.path.basename(
        doc.metadata['source']).replace('_investorcom_1.pdf', ''
   )

In [31]:
doc.metadata

{'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/google_investorcom_1.pdf',
 'company': 'google'}

In [16]:
#####

In [97]:
entity_types = ['ORG', 'PERSON']

In [98]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [125]:
doc_metadata={}

In [116]:
# multiple values for each entity

In [100]:
doc=nlp(chunked_data[0].page_content)

In [101]:
doc_entities = set([
    (X.label_, X.text) for X in doc.ents if any(
        k in X.label_ for k in entity_types
    )
])

In [102]:
for e, v in doc_entities :
    doc_metadata.setdefault(e, []).append(v)

In [103]:
doc_metadata

{'ORG': ['Google',
  'Amazon',
  'Amazon Web Services',
  'Heightened Focus On Cost Savings',
  'CFRA',
  'Amazon Jumps Into Arti\x00cial Intelligence',
  'Amazon Health Services',
  'AI',
  'One Medical',
  'Oppenheimer',
  'Microsoft',
  'AWS',
  'MSFT'],
 'PERSON': ['Jason Helfstein',
  'Twitch',
  'Andy Jassy',
  'Sundaram',
  'Jassy',
  'Neil Lindsay',
  'Arun Sundaram']}

In [121]:
# unique value for each entity

In [143]:
doc=nlp(chunked_data[0].page_content)
x = Counter([
    (X.text, X.label_) for X in doc.ents if any(
        k in X.label_ for k in entity_types
    )
])

In [152]:
doc_metadata = dict.fromkeys(set([_[1] for _ in x.keys()]))
for v in list(dict(x.keys()).items()):
    if None in doc_metadata.values():
        if doc_metadata[v[1]] is None:
            doc_metadata[v[1]]=[v[0]]
    else:
        break

In [154]:
doc_metadata = dict.fromkeys(set([_[1] for _ in x.keys()]))

In [156]:
doc_metadata['ORG']='Ama'

In [33]:
doc=nlp('what are the pros and cons of investing in Amazon, Google')
print([(X.text, X.label_) for X in doc.ents])

[('Amazon', 'ORG'), ('Google', 'ORG')]


In [ ]:
######

##### - Store

In [ ]:
chunks_max = 15

In [ ]:
vectorstore_engine = 'Finbot-embedding-2'

In [ ]:
embedding_model = OpenAIEmbeddings(deployment=vectorstore_engine)

In [ ]:
try:
    vectorstore.delete_collection()
except (ValueError, NameError):
    pass
for _ in range(0, len(chunked_data), chunks_max):
    vectorstore = Chroma.from_documents(
        documents=chunked_data[_: _ + chunks_max], embedding=embedding_model
    )

In [ ]:
len(vectorstore.get()['documents'])

In [ ]:
vectorstore.get().keys()

In [ ]:
vectorstore.get()['metadatas']

##### - Retrieve

In [ ]:
question = "tell me about the world in brief?"
docs = vectorstore.similarity_search_with_relevance_scores(question, k=7)
len(docs)

In [ ]:
len(docs[0][0].page_content)

In [ ]:
# print(docs[1][0].page_content)

##### - Generate

In [ ]:
llm_engine = 'finbot-gpt'
llm_model = 'text-davinci-002'
temperature = 0
search_type = 'mmr'
retrieval_kwargs = {'k': 5, 'lambda_mult': 0.5, 'fetch_k':10}
answer_max_tokens = 512
save_folder = '../data/pdfs/'
prompt_template_file = os.path.join(os.getcwd(), '../scripts/prompt_template.txt')
pdf_list = [
    _ for _ in glob.glob(os.path.join(os.getcwd(), save_folder, '*.pdf'))
]
web_list = []
langchain.debug=False

In [ ]:
############

In [ ]:
base_llm = AzureOpenAI(
    engine=llm_engine, 
    model_name=llm_model, 
    temperature=temperature, 
    max_tokens=answer_max_tokens
)

In [ ]:
langchain.debug=False

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
condense_question_template = os.path.join(
    os.getcwd(), '..', 'scripts/condense_question_template.txt'
)
condense_question_input = c.condense_question_input

In [ ]:
chat_prompt = PromptTemplate.from_file(
    condense_question_template,
    input_variables=condense_question_input
)

In [ ]:
qna_prompt_template = os.path.join(os.getcwd(), '..', 'scripts/qna_prompt_template.txt')
qna_prompt_input = c.qna_prompt_input
prompt_role = c.prompt_role

In [ ]:
qna_prompt = PromptTemplate.from_file(
    qna_prompt_template,
    input_variables=qna_prompt_input,
    partial_variables={'role':prompt_role}
)

In [ ]:
history_tokens = 2000
memory = ConversationSummaryBufferMemory(
    llm=base_llm, 
    memory_key="chat_history", 
    return_messages=True, 
    max_token_limit=history_tokens
)

In [ ]:
qna_chain = ConversationalRetrievalChain.from_llm(
    base_llm, 
    retriever=vectorstore.as_retriever(
        search_type=search_type, search_kwargs=retrieval_kwargs
    ),
    memory=memory,
    condense_question_prompt=chat_prompt,
    combine_docs_chain_kwargs=dict(prompt=qna_prompt)
)

In [ ]:
question = "tell me about the world in brief?"
results = qna_chain({"question": question})

In [ ]:
print(results['answer'])